# PT data

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2024-04-08 14:59:24.507035: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 14:59:24.527683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 14:59:24.527701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 14:59:24.528141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 14:59:24.531541: I tensorflow/core/platform/cpu_feature_guar

In [2]:
GLC24_PA_metadata_train = pd.read_pickle("processed_data/GLC24_PA_metadata_train_elevation.pkl")

In [8]:
with pd.option_context('use_inf_as_na', True):
    display(GLC24_PA_metadata_train.isna().sum())

/tmp/ipykernel_19746/421287792.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('use_inf_as_na', True):


lon                       0
lat                       0
year                      0
geoUncertaintyInM     12496
areaInM2             234864
region                    0
country                   0
speciesId                 0
surveyId                  0
elevation                 0
dtype: int64

In [10]:
GLC24_PA_metadata_train = GLC24_PA_metadata_train.drop(columns=['geoUncertaintyInM'])

KeyError: "['geoUncertaintyInM'] not found in axis"

In [16]:
with pd.option_context('use_inf_as_na', True):
    GLC24_PA_metadata_train.areaInM2 = GLC24_PA_metadata_train.areaInM2.fillna(GLC24_PA_metadata_train.areaInM2.mean())

/tmp/ipykernel_19746/3152395630.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('use_inf_as_na', True):


In [17]:
with pd.option_context('use_inf_as_na', True):
    display(GLC24_PA_metadata_train.isna().sum())

/tmp/ipykernel_19746/421287792.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('use_inf_as_na', True):


lon          0
lat          0
year         0
areaInM2     0
region       0
country      0
speciesId    0
surveyId     0
elevation    0
dtype: int64

In [24]:
GLC24_PA_metadata_train.head()

,lon,lat,year,areaInM2,region,country,speciesId,surveyId,elevation
0,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,6874,212,26.0
1,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,476,212,26.0
2,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,11157,212,26.0
3,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,8784,212,26.0
4,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,4530,212,26.0


In [23]:
GLC24_PA_metadata_train.speciesId = GLC24_PA_metadata_train.speciesId.astype('uint16')

In [29]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(dtype='uint8')

In [30]:
transformed = ohe.fit_transform(GLC24_PA_metadata_train[['region', 'country']])


In [34]:
print(transformed.toarray().shape)

(1483637, 37)


In [35]:
GLC24_PA_metadata_train.shape

(1483637, 9)

In [44]:
np.concatenate((ohe.categories_[0], ohe.categories_[1]), axis=0)

array(['ALPINE', 'ATLANTIC', 'BLACK SEA', 'BOREAL', 'CONTINENTAL',
       'MEDITERRANEAN', 'PANNONIAN', 'STEPPIC', 'Andorra', 'Austria',
       'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia',
       'Czech Republic', 'Denmark', 'France', 'Germany', 'Greece',
       'Hungary', 'Ireland', 'Italy', 'Latvia', 'Luxembourg', 'Monaco',
       'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
       'Switzerland', 'The former Yugoslav Republic of Macedonia'],
      dtype=object)

In [45]:
GLC24_PA_metadata_train[np.concatenate((ohe.categories_[0], ohe.categories_[1]), axis=0)] = transformed.toarray()
print(GLC24_PA_metadata_train.head())

        lon        lat  year  areaInM2         region country  speciesId  \
0  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France       6874   
1  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France        476   
2  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France      11157   
3  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France       8784   
4  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France       4530   

   surveyId  elevation  ALPINE  ...  Norway  Poland  Portugal  Romania  \
0       212       26.0       0  ...       0       0         0        0   
1       212       26.0       0  ...       0       0         0        0   
2       212       26.0       0  ...       0       0         0        0   
3       212       26.0       0  ...       0       0         0        0   
4       212       26.0       0  ...       0       0         0        0   

   Serbia  Slovakia  Slovenia  Spain  Switzerland  \
0       0         0         0      0         

In [46]:
GLC24_PA_metadata_train = GLC24_PA_metadata_train.drop(columns=['region', 'country'])

In [47]:
# save GLC24_PA_metadata_train pickled
GLC24_PA_metadata_train.to_pickle("processed_data/GLC24_PA_metadata_train_elevation_onehot.pkl")